<a href="https://colab.research.google.com/github/veronica1908/MARKETING_E2_A_III/blob/main/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Instalar la extensión ipython-sql
!pip install -q ipython-sql

# Cargar la extensión
%load_ext sql

# Instalar bibliotecas necesarias
!pip install sqlalchemy
!pip install mysql-connector-python

# Importar bibliotecas
import sqlite3
from sqlalchemy import create_engine

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 45.0 MB/s eta 0:00:00


In [8]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

In [14]:
#Crear tabla con usuarios con más de 50 películas vistas y menos de 1000

# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear tabla con usuarios con más de 50 películas vistas y menos de 1000
query = """
    DROP TABLE IF EXISTS usuarios_sel;
    CREATE TABLE usuarios_sel AS
    SELECT "userId" AS user_id, COUNT(*) AS cnt_rat
    FROM ratings
    GROUP BY "userId"
    HAVING cnt_rat > 50 AND cnt_rat <= 1000
    ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)

# Cerrar la conexión
conn.close()

In [16]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear tabla con películas que han sido calificadas por más de 50 usuarios
query = """
DROP TABLE IF EXISTS movies_sel;

CREATE TABLE movies_sel AS
SELECT movieId,
       COUNT(*) AS cnt_rat
FROM ratings
GROUP BY movieId
HAVING cnt_rat > 50
ORDER BY cnt_rat DESC;
"""

# Ejecutar la consulta
conn.executescript(query)

# Cerrar la conexión
conn.close()

In [19]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')

# Crear tablas filtradas de calificaciones con usuarios y películas
##ratings_final

query = """
DROP TABLE IF EXISTS ratings_final;

CREATE TABLE ratings_final AS
SELECT a."userId" AS user_id,
       a.movieId AS movie_id,
       a.rating AS rating
FROM ratings a
INNER JOIN movies_sel b ON a.movieId = b.movieId
INNER JOIN usuarios_sel c ON a."userId" = c.user_id;
"""

##movies_final
"""
DROP TABLE IF EXISTS movies_final;

CREATE TABLE movies_final AS
SELECT a.movieId AS movie_id,
       a.title AS movie_title,
       a.genres AS genres
FROM movies a
INNER JOIN movies_sel c ON a.movieId = c.movieId;
"""

# Ejecutar la consulta
conn.executescript(query)

# Cerrar la conexión
conn.close()

In [26]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_movies')
cursor = conn.cursor()

# Script SQL
script = """
DROP TABLE IF EXISTS full_ratings;

CREATE TABLE full_ratings AS
SELECT a.*,
       c.movie_title,
       c.genres
FROM ratings_final a
INNER JOIN movies_final c ON a.movie_id = c.movie_id;
"""

# Ejecutar el script SQL
cursor.executescript(script)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()

print("Tabla 'full_ratings' creada con éxito.")

Tabla 'full_ratings' creada con éxito.


# Exportar la tabla preprocesada en SQL

In [27]:
# Conectar a la base de datos
conn = sqlite3.connect('db_movies')

# Consulta SQL para seleccionar todos los datos de la tabla full_ratings
query = "SELECT * FROM full_ratings"

# Leer los datos en un DataFrame de pandas
df = pd.read_sql_query(query, conn)

# Guardar el DataFrame como un archivo CSV
df.to_csv('full_ratings.csv', index=False)

# Cerrar la conexión
conn.close()

print("La tabla 'full_ratings' se ha exportado como full_ratings.csv")

La tabla 'full_ratings' se ha exportado como full_ratings.csv


In [28]:
from google.colab import files

files.download('full_ratings.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>